# PieceMatcher sample

Use the `PieceMatcher`


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from functools import partial

from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.data_models.segment_id import SegmentId
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.image.utils import draw_contour
from snap_fit.image.utils import show_image_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece_matcher import PieceMatcher
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager


## Params and config


In [ ]:
sf_paths = get_snap_fit_paths()
rprint(sf_paths)

### Load oca data


In [ ]:
# # 1. Configure ArUco Board and Detector
# # Using defaults which match the printed board used for 'data/oca'
# board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
# detector_config = ArucoDetectorConfig(board=board_config)

# # 2. Initialize SheetAruco helper
# # crop_margin is automatically calculated from the detector configuration
# sheet_aruco = SheetAruco(detector_config)

# # 3. Define the loader function
# # SheetAruco.load_sheet handles loading, rectification, and Sheet creation
# aruco_loader = sheet_aruco.load_sheet

# # 4. define base folder
# paths = get_snap_fit_paths()
# data_dir = paths.data_fol / "oca"
# lg.info(f"Loading data from {data_dir}")

# # 5. instantiate manager and load
# manager = SheetManager()
# manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(
    markers_x=7,
    markers_y=5,
    marker_length=100,
    marker_separation=100,
)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = partial(sheet_aruco.load_sheet, min_area=5_000)

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "sample_puzzle_v1"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(
    folder_path=data_dir,
    pattern="*.png",
    loader_func=aruco_loader,
)

## Debug loaded sheets


In [ ]:
sheets = manager.get_sheets_ls()
len(sheets)

sheet = sheets[0]
print(sheet.img_fp)
show_image_mpl(sheet.img_bw)


In [ ]:
pieces = manager.get_pieces_ls()
len(pieces)

## Piece match


In [ ]:
# 1. Initialize the PieceMatcher
matcher = PieceMatcher(manager)

# 2. Run matching for all segments
# This will compare every segment with segments from other pieces
matcher.match_all()

# 3. Get and display the top 5 matches
top_matches = matcher.get_top_matches(5)
rprint("[bold green]Top 5 Matches:[/bold green]")
for i, res in enumerate(top_matches, 1):
    rprint(f"{i}. {res.seg_id1} <-> {res.seg_id2} | Similarity: {res.similarity:.4f}")

# 4. Demonstrate querying matches for a specific piece
# Let's pick the first piece from the first sheet
all_ids = manager.get_segment_ids_all()
if all_ids:
    sample_id = all_ids[0]
    piece_matches = matcher.get_matches_for_piece(sample_id.piece_id)

    rprint(
        "\n[bold blue]Matches for piece "
        f"{sample_id.sheet_id}:{sample_id.piece_id}:[/bold blue]"
    )
    # Show top 3 matches for this piece
    piece_matches.sort(key=lambda x: x.similarity)
    for res in piece_matches[:3]:
        other = res.get_other(
            SegmentId(
                piece_id=sample_id.piece_id,
                edge_pos=res.seg_id1.edge_pos
                if res.seg_id1.piece_id == sample_id.piece_id
                else res.seg_id2.edge_pos,
            )
        )
        # Note: get_other needs the exact SegmentId.
        # Let's simplify the display:
        rprint(
            f"  - {res.seg_id1} <-> {res.seg_id2} | Similarity: {res.similarity:.4f}"
        )


In [ ]:
# visualize pieces overlapping using the best match found above

if len(top_matches) == 0:
    msg = "No matches found to visualize."
    raise ValueError(msg)

best = top_matches[3]

# Get pieces and segments
p1 = manager.get_piece_by_segment_id(best.seg_id1)
p2 = manager.get_piece_by_segment_id(best.seg_id2)

seg1 = manager.get_segment(best.seg_id1)
seg2 = manager.get_segment(best.seg_id2)

if not (p1 and p2 and seg1 and seg2):
    msg = "Could not find pieces or segments for the best match."
    raise ValueError(msg)

rprint(f"Visualizing overlap for: {best.seg_id1} and {best.seg_id2}")

# Create a segment matcher to get the transformation
seg_match = SegmentMatcher(seg1, seg2)
sim = seg_match.compute_similarity()
rprint(f"Similarity: {sim:.4f}")

# Draw the segments on piece 2's image
# We dim the background image for better visibility
p2_img = p2.img_bw.copy() // 4

# Draw target segment (seg2)
draw_contour(p2_img, seg_match.s2.points, color=150)

# Draw transformed source segment (seg1)
draw_contour(
    p2_img,
    seg_match.s1_points_transformed,
    color=255,
)

show_image_mpl(p2_img, figsize=(8, 8))


In [ ]:
# show p1 and p2 side by side, original images
if not (p1 and p2 and seg1 and seg2):
    msg = "Could not find pieces or segments for the best match."
    raise ValueError(msg)
show_image_mpl(p1.img_orig, figsize=(10, 5))
show_image_mpl(p2.img_orig, figsize=(10, 5))